In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important;} </style>"))

In [2]:
import os
import datetime
import sys
import glob
import random
import pickle

import numpy         as np
import pandas        as pd

from scipy.fftpack   import fft
from pydub           import AudioSegment
from IPython.display import Audio

from pydub.silence   import split_on_silence
from pydub.silence   import detect_leading_silence

from tqdm            import tqdm

In [3]:
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
random.seed(42)

<h1 style="background-color:LightGreen;"> <center> <a id='start_cell'></a> Table Of Contents </center></h1>

[Create Datasets](#create_dataset) </br>

<h1 style="background-color:#3cA8EF;"> <center> <a id='create_dataset'></a> Choose Files </center> </h1>

<h1 style="color:red"> Choose speakers with at least 10 files </h1>

In [4]:
if sys.platform != "win32":
    # linux    
    dataset_path = r"/home/amitli/Datasets/CV_14/cv-corpus-14.0-2023-06-23-ru/cv-corpus-14.0-2023-06-23/ru/"                  
else:
    # windows    
    dataset_path = r"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\ru"

In [5]:
CHOOSE_FILES = False
    
if CHOOSE_FILES is True:   
    train_df = pd.read_csv(dataset_path + "/train.tsv", delimiter="\t")
    test_df  = pd.read_csv(dataset_path + "/test.tsv",  delimiter="\t")
    df_all   = pd.concat([train_df, test_df])
    
    grouped         = df_all.groupby('client_id')
    filtered_groups = grouped.filter(lambda x: len(x) >= 10)
    df              = pd.DataFrame(filtered_groups)
    
    df.to_csv("cv_14_walkie_ru.csv")
else:
    df = pd.read_csv("cv_14_walkie_ru.csv")

In [6]:
print(f"Number of speakers: {len(set(df.client_id.values))}")
print(f"Dataframe size    : {len(df)}")

Number of speakers: 527
Dataframe size    : 29415


<h1 style="background-color:#3cA8EF;"> <center> <a id='player'></a> Player </center> </h1>

In [7]:
from pygame import mixer
import time
import IPython
import os

pygame 2.1.0 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
num_to_freq_file = {}

num_to_freq_file[0]  = 'encoded_tones/audiocheck.net_sin_500Hz_-3dBFS_1.5s.wav'
num_to_freq_file[1]  = 'encoded_tones/audiocheck.net_sin_1000Hz_-10dBFS_1s.wav'
num_to_freq_file[2]  = 'encoded_tones/audiocheck.net_sin_1100Hz_-10dBFS_1s.wav'
num_to_freq_file[3]  = 'encoded_tones/audiocheck.net_sin_1200Hz_-10dBFS_1s.wav'
num_to_freq_file[4]  = 'encoded_tones/audiocheck.net_sin_1300Hz_-10dBFS_1s.wav'
num_to_freq_file[5]  = 'encoded_tones/audiocheck.net_sin_1400Hz_-10dBFS_1s.wav'
num_to_freq_file[6]  = 'encoded_tones/audiocheck.net_sin_1500Hz_-10dBFS_1s.wav'
num_to_freq_file[7]  = 'encoded_tones/audiocheck.net_sin_1600Hz_-10dBFS_1s.wav'
num_to_freq_file[8]  = 'encoded_tones/audiocheck.net_sin_1700Hz_-10dBFS_1s.wav'
num_to_freq_file[9]  = 'encoded_tones/audiocheck.net_sin_1800Hz_-10dBFS_1s.wav'
num_to_freq_file[10] = 'encoded_tones/audiocheck.net_sin_1900Hz_-10dBFS_1s.wav'

for f in num_to_freq_file.values():
    if not os.path.exists(f):
        print(f"Missing encoding file: {f}")    

In [9]:
def play_file(filename, fileLengthInSeconds):
    mixer.music.load(filename)
    mixer.music.play()
    time.sleep(fileLengthInSeconds)    
    while mixer.music.get_busy():  
        print("\n---> busy - wait -----\n")
        time.sleep(0.05)
    #time.sleep(0.05)

In [10]:
def play_beep(file, start_time):

    # log start
    end_time = time.time()
    str_log   = f"\nStart play beep, Elapse: {round(end_time-start_time, 3)}"        
    print(str_log)
    file.write(str_log)
    
    # play beep
    play_file(num_to_freq_file[1], 1)
    
    # log end
    end_time = time.time()
    str_log   = f"\nEnd play beep, Elapse: {round(end_time-start_time, 3)}"        
    print(str_log)
    file.write(str_log)                    

In [11]:
def play_language(df, dataset_path,  start_from_client=0):
        
    current_date = datetime.datetime.now().strftime('%d_%m_%Y_%H_%M_%S')        
    all_clients  = list(set(df.client_id.values))

    mixer.init()     
    start_time = time.time()        
    with open(f"log_{current_date}.txt", "w") as f:
        
        play_beep(f, start_time)
        time.sleep(0.05)  
        play_beep(f, start_time)
        time.sleep(0.05)  
        
        for c_i in range(len(all_clients)):  
            
            if c_i < start_from_client:
                continue
            
            client_id    = all_clients[c_i]
            df_client    = df[df.client_id == client_id]            
            client_files = df_client.path.values
                                                            
            str_log   = f"\n\nClient_Id: {client_id} [{c_i+1}/{len(all_clients)}] Files: {len(client_files)}"        
            print(str_log)
            f.write(str_log)
                          
            for file_i in range(len(client_files)):      
                
                file         = client_files[file_i]
                fullFilePath = f"{dataset_path}/clips/{file}"
                fileLength   = AudioSegment.from_mp3(fullFilePath).duration_seconds
                           
                end_time = time.time()
                str_log = f"\n\t[{file_i+1}/{len(client_files)}] Start Play file: {file}, Elapse: {round(end_time-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                
                play_beep(f, start_time)
                time.sleep(0.05)  
                play_file(fullFilePath, fileLength)
                time.sleep(3)  
                end_start = time.time()

                str_log = f"\n\t[{file_i+1}/{len(client_files)}] ---> Played file: {file}, length: {fileLength} seconds, Elapse: {round(end_start-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                #time.sleep(0.2)  
                  
            #time.sleep(0.2)  
            
        end_time = time.time()        
        str_log = f"\nClose File, Elapse: {round(end_time-start_time, 3)}"
        print(str_log)
        f.write(str_log)

        f.close()
        

In [ ]:
play_language(df, dataset_path, start_from_client=100)        


Start play beep, Elapse: 0.0

End play beep, Elapse: 1.008

Start play beep, Elapse: 1.058

End play beep, Elapse: 2.06


Client_Id: fc748a5dd32c662d0fd3d30ede650a73f5d7b13eded83b3785b2756b3bb216a5850c85880b76c66fbc6e05362240965bd2816c8dabc307bb620d23f11cd7b334 [101/527] Files: 10

	[1/10] Start Play file: common_voice_ru_21107819.mp3, Elapse: 2.2

Start play beep, Elapse: 2.2

End play beep, Elapse: 3.201

	[1/10] ---> Played file: common_voice_ru_21107819.mp3, length: 6.144 seconds, Elapse: 12.405

	[2/10] Start Play file: common_voice_ru_21107820.mp3, Elapse: 12.49

Start play beep, Elapse: 12.49

End play beep, Elapse: 13.492

	[2/10] ---> Played file: common_voice_ru_21107820.mp3, length: 5.784 seconds, Elapse: 22.335

	[3/10] Start Play file: common_voice_ru_21107821.mp3, Elapse: 22.413

Start play beep, Elapse: 22.413

End play beep, Elapse: 23.414

	[3/10] ---> Played file: common_voice_ru_21107821.mp3, length: 3.096 seconds, Elapse: 29.566

	[4/10] Start Play file: common_voi


	[13/131] ---> Played file: common_voice_ru_25679208.mp3, length: 2.7 seconds, Elapse: 328.313

	[14/131] Start Play file: common_voice_ru_25679353.mp3, Elapse: 328.379

Start play beep, Elapse: 328.379

End play beep, Elapse: 329.381

	[14/131] ---> Played file: common_voice_ru_25679353.mp3, length: 3.24 seconds, Elapse: 335.678

	[15/131] Start Play file: common_voice_ru_25679355.mp3, Elapse: 335.753

Start play beep, Elapse: 335.753

End play beep, Elapse: 336.753

	[15/131] ---> Played file: common_voice_ru_25679355.mp3, length: 3.24 seconds, Elapse: 343.05

	[16/131] Start Play file: common_voice_ru_25679971.mp3, Elapse: 343.126

Start play beep, Elapse: 343.126

End play beep, Elapse: 344.127

	[16/131] ---> Played file: common_voice_ru_25679971.mp3, length: 7.704 seconds, Elapse: 354.893

	[17/131] Start Play file: common_voice_ru_25679976.mp3, Elapse: 354.981

Start play beep, Elapse: 354.981

End play beep, Elapse: 355.982

	[17/131] ---> Played file: common_voice_ru_25679976


End play beep, Elapse: 626.48

	[48/131] ---> Played file: common_voice_ru_25692075.mp3, length: 4.356 seconds, Elapse: 633.894

	[49/131] Start Play file: common_voice_ru_25692087.mp3, Elapse: 633.974

Start play beep, Elapse: 633.974

End play beep, Elapse: 634.975

	[49/131] ---> Played file: common_voice_ru_25692087.mp3, length: 6.516 seconds, Elapse: 644.551

	[50/131] Start Play file: common_voice_ru_25692098.mp3, Elapse: 644.629

Start play beep, Elapse: 644.629

End play beep, Elapse: 645.63

	[50/131] ---> Played file: common_voice_ru_25692098.mp3, length: 2.988 seconds, Elapse: 651.674

	[51/131] Start Play file: common_voice_ru_25692117.mp3, Elapse: 651.748

Start play beep, Elapse: 651.748

End play beep, Elapse: 652.749

	[51/131] ---> Played file: common_voice_ru_25692117.mp3, length: 3.384 seconds, Elapse: 659.191

	[52/131] Start Play file: common_voice_ru_25692134.mp3, Elapse: 659.269

Start play beep, Elapse: 659.269

End play beep, Elapse: 660.27

	[52/131] ---> Pla


End play beep, Elapse: 916.281

	[83/131] ---> Played file: common_voice_ru_25693723.mp3, length: 3.132 seconds, Elapse: 922.47

	[84/131] Start Play file: common_voice_ru_25693747.mp3, Elapse: 922.536

Start play beep, Elapse: 922.537

End play beep, Elapse: 923.538

	[84/131] ---> Played file: common_voice_ru_25693747.mp3, length: 3.204 seconds, Elapse: 929.797

	[85/131] Start Play file: common_voice_ru_25693776.mp3, Elapse: 929.864

Start play beep, Elapse: 929.864

End play beep, Elapse: 930.865

	[85/131] ---> Played file: common_voice_ru_25693776.mp3, length: 4.032 seconds, Elapse: 937.954

	[86/131] Start Play file: common_voice_ru_25693833.mp3, Elapse: 938.019

Start play beep, Elapse: 938.019

End play beep, Elapse: 939.021

	[86/131] ---> Played file: common_voice_ru_25693833.mp3, length: 3.132 seconds, Elapse: 945.21

	[87/131] Start Play file: common_voice_ru_25693875.mp3, Elapse: 945.289

Start play beep, Elapse: 945.289

End play beep, Elapse: 946.29

	[87/131] ---> Pla


End play beep, Elapse: 1211.185

	[117/131] ---> Played file: common_voice_ru_25695317.mp3, length: 5.004 seconds, Elapse: 1219.247

	[118/131] Start Play file: common_voice_ru_25695338.mp3, Elapse: 1219.327

Start play beep, Elapse: 1219.328

End play beep, Elapse: 1220.329

	[118/131] ---> Played file: common_voice_ru_25695338.mp3, length: 4.644 seconds, Elapse: 1228.031

	[119/131] Start Play file: common_voice_ru_25695371.mp3, Elapse: 1228.113

Start play beep, Elapse: 1228.113

End play beep, Elapse: 1229.115

	[119/131] ---> Played file: common_voice_ru_25695371.mp3, length: 5.544 seconds, Elapse: 1237.718

	[120/131] Start Play file: common_voice_ru_25695419.mp3, Elapse: 1237.782

Start play beep, Elapse: 1237.782

End play beep, Elapse: 1238.783

	[120/131] ---> Played file: common_voice_ru_25695419.mp3, length: 4.5 seconds, Elapse: 1246.342

	[121/131] Start Play file: common_voice_ru_25695502.mp3, Elapse: 1246.411

Start play beep, Elapse: 1246.411

End play beep, Elapse: 12

...


End play beep, Elapse: 4138.831

	[42/44] ---> Played file: common_voice_ru_21930515.mp3, length: 7.368 seconds, Elapse: 4149.261

	[43/44] Start Play file: common_voice_ru_21930516.mp3, Elapse: 4149.348

Start play beep, Elapse: 4149.348

End play beep, Elapse: 4150.349

	[43/44] ---> Played file: common_voice_ru_21930516.mp3, length: 9.12 seconds, Elapse: 4162.533

	[44/44] Start Play file: common_voice_ru_21930517.mp3, Elapse: 4162.612

Start play beep, Elapse: 4162.612

End play beep, Elapse: 4163.614

	[44/44] ---> Played file: common_voice_ru_21930517.mp3, length: 6.456 seconds, Elapse: 4173.13


Client_Id: 37b0cf86f60c4c35a3a8180f2a7216612930de0db3722be1c9e4e41ee2a2b9c6284593925ee1f56567c4195e899a099fb976bd9a0bdd7f5419bcd6cd59e6037b [109/527] Files: 13

	[1/13] Start Play file: common_voice_ru_25190413.mp3, Elapse: 4173.199

Start play beep, Elapse: 4173.199

End play beep, Elapse: 4174.2

	[1/13] ---> Played file: common_voice_ru_25190413.mp3, length: 3.78 seconds, Elapse: 418


End play beep, Elapse: 4465.475

	[8/170] ---> Played file: common_voice_ru_33289221.mp3, length: 5.688 seconds, Elapse: 4474.221

	[9/170] Start Play file: common_voice_ru_33289222.mp3, Elapse: 4474.309

Start play beep, Elapse: 4474.309

End play beep, Elapse: 4475.31

	[9/170] ---> Played file: common_voice_ru_33289222.mp3, length: 4.968 seconds, Elapse: 4483.337

	[10/170] Start Play file: common_voice_ru_33289228.mp3, Elapse: 4483.416

Start play beep, Elapse: 4483.416

End play beep, Elapse: 4484.418

	[10/170] ---> Played file: common_voice_ru_33289228.mp3, length: 7.596 seconds, Elapse: 4495.075

	[11/170] Start Play file: common_voice_ru_33289229.mp3, Elapse: 4495.151

Start play beep, Elapse: 4495.152

End play beep, Elapse: 4496.153

	[11/170] ---> Played file: common_voice_ru_33289229.mp3, length: 7.02 seconds, Elapse: 4506.232

	[12/170] Start Play file: common_voice_ru_33289230.mp3, Elapse: 4506.305

Start play beep, Elapse: 4506.305

End play beep, Elapse: 4507.306

	[1


End play beep, Elapse: 4799.211

	[42/170] ---> Played file: common_voice_ru_33290264.mp3, length: 4.176 seconds, Elapse: 4806.443

	[43/170] Start Play file: common_voice_ru_33290267.mp3, Elapse: 4806.521

Start play beep, Elapse: 4806.521

End play beep, Elapse: 4807.522

	[43/170] ---> Played file: common_voice_ru_33290267.mp3, length: 7.02 seconds, Elapse: 4817.602

	[44/170] Start Play file: common_voice_ru_33290285.mp3, Elapse: 4817.679

Start play beep, Elapse: 4817.679

End play beep, Elapse: 4818.681

	[44/170] ---> Played file: common_voice_ru_33290285.mp3, length: 4.428 seconds, Elapse: 4826.166

	[45/170] Start Play file: common_voice_ru_33290286.mp3, Elapse: 4826.243

Start play beep, Elapse: 4826.243

End play beep, Elapse: 4827.244

	[45/170] ---> Played file: common_voice_ru_33290286.mp3, length: 5.688 seconds, Elapse: 4835.992

	[46/170] Start Play file: common_voice_ru_33290287.mp3, Elapse: 4836.069

Start play beep, Elapse: 4836.07

End play beep, Elapse: 4837.071




End play beep, Elapse: 5126.904

	[76/170] ---> Played file: common_voice_ru_33291179.mp3, length: 4.32 seconds, Elapse: 5134.281

	[77/170] Start Play file: common_voice_ru_33291181.mp3, Elapse: 5134.347

Start play beep, Elapse: 5134.347

End play beep, Elapse: 5135.348

	[77/170] ---> Played file: common_voice_ru_33291181.mp3, length: 3.96 seconds, Elapse: 5142.366

	[78/170] Start Play file: common_voice_ru_33291184.mp3, Elapse: 5142.441

Start play beep, Elapse: 5142.441

End play beep, Elapse: 5143.443

	[78/170] ---> Played file: common_voice_ru_33291184.mp3, length: 4.896 seconds, Elapse: 5151.397

	[79/170] Start Play file: common_voice_ru_33291251.mp3, Elapse: 5151.481

Start play beep, Elapse: 5151.481

End play beep, Elapse: 5152.482

	[79/170] ---> Played file: common_voice_ru_33291251.mp3, length: 5.616 seconds, Elapse: 5161.155

	[80/170] Start Play file: common_voice_ru_33291253.mp3, Elapse: 5161.23

Start play beep, Elapse: 5161.23

End play beep, Elapse: 5162.231

	[


End play beep, Elapse: 5469.782

	[110/170] ---> Played file: common_voice_ru_33292063.mp3, length: 5.868 seconds, Elapse: 5478.709

	[111/170] Start Play file: common_voice_ru_33292064.mp3, Elapse: 5478.78

Start play beep, Elapse: 5478.78

End play beep, Elapse: 5479.781

	[111/170] ---> Played file: common_voice_ru_33292064.mp3, length: 5.328 seconds, Elapse: 5488.165

	[112/170] Start Play file: common_voice_ru_33292065.mp3, Elapse: 5488.24

Start play beep, Elapse: 5488.24

End play beep, Elapse: 5489.242

	[112/170] ---> Played file: common_voice_ru_33292065.mp3, length: 4.788 seconds, Elapse: 5497.085

	[113/170] Start Play file: common_voice_ru_33292066.mp3, Elapse: 5497.161

Start play beep, Elapse: 5497.161

End play beep, Elapse: 5498.163

	[113/170] ---> Played file: common_voice_ru_33292066.mp3, length: 6.516 seconds, Elapse: 5507.735

	[114/170] Start Play file: common_voice_ru_33292136.mp3, Elapse: 5507.811

Start play beep, Elapse: 5507.811

End play beep, Elapse: 5508

<h1 style="background-color:#3cA8EF;"> <center><a id='parseLog'></a> Parse player log </center> </h1>

In [ ]:
def parse_log(log_file):
    
    arr_clients            = []   # arr of client_id [may be duplicate - depends on num of files per client]
    arr_cv                 = []   # cv file
    arr_time               = []   # cv file length    
    arr_num_cv_per_speaker = []   # for each speaker - number of speechs
    arr_start_speech_time  = []   # for each speech - start time
    arr_end_speech_time    = []

    with open(log_file) as f:
        lines = f.readlines()

        counter     = 0
        last_client = None
        start_time  = None        
        
        for line in lines:

            if line.find("Client_Id") != -1:
                
                client      = line[line.find(": ")+2 : line.find(" [")]
                last_client = client
                if counter > 0:
                    arr_num_cv_per_speaker.append(counter) 
                    counter = 0
                    
                 
            elif line.find("Start Play file:") != -1:                     
                start_time = float(line[line.find("Elapse:")+8:])                

                    
            elif line.find("---> Played file") != -1:                
                cv_file = line[line.find(": c")+2 : line.find(", ")]
                length  = float(line[line.find("length:")+8 : line.find(" seconds")])                
                end_t   = float(line[line.find("Elapse: ")+8:])                
                counter = counter + 1
                
                arr_clients.append(last_client)
                arr_cv.append(cv_file)
                arr_time.append(length)                
                arr_end_speech_time.append(end_t)
                if start_time is not None:
                    arr_start_speech_time.append(start_time)
                    start_time = None
                else:
                    print(f"Error, start_time is None")
           
            
    arr_num_cv_per_speaker.append(counter)         
    print(f"Total number of speakers: {len(set(arr_clients))}, files: {len(set(arr_cv))}, langs: {len(set(arr_lang))}")
    print(f"len(arr_start_speech_time) = {len(arr_start_speech_time)}")
    return arr_clients, arr_cv, arr_time,  arr_num_cv_per_speaker, arr_start_speech_time, arr_end_speech_time

In [ ]:
NIGHT_RUN_1 = r"C:\Users\amitli\Repo\WalkieTalkieRecorder\log_01_04_2024_15_03_43.txt"
arr_clients, arr_cv, arr_time,  arr_num_cv_per_speaker, arr_start_speech_time, arr_end_speech_time = parse_log(NIGHT_RUN_1)

<h1 style="background-color:#3cA8EF;"> <center> <a id='Reciever'></a> Reciever </center> </h1>

In [ ]:
from dataclasses        import dataclass, asdict
from concurrent.futures import ThreadPoolExecutor
from pydub              import AudioSegment,silence

import datetime
import pyaudio
import wave

In [ ]:

@dataclass
class StreamParams:
    format: int            = pyaudio.paInt16
    channels: int          = 1
    rate: int              = 8000
    frames_per_buffer: int = 1024
    input: bool            = True
    output: bool           = False

    def to_dict(self) -> dict:
        return asdict(self)

class Recorder:
    """Recorder uses the blocking I/O facility from pyaudio to record sound
    from mic.
    Attributes:
        - stream_params: StreamParams object with values for pyaudio Stream
            object
    """
    def __init__(self, stream_params: StreamParams) -> None:
        self.stream_params = stream_params
        self._pyaudio      = None
        self._stream       = None
        self._wav_file     = None
        self._counter      = 0

    def record(self, duration: int, save_path: str, num_files_to_create: int) -> None:
        """Record sound from mic for a given amount of seconds.
        :param duration: Number of seconds we want to record for
        :param save_path: Where to store recording
        """
        print("Start recording...")
        self.save_path = save_path
        self._create_recording_resources()
        self._write_wav_file_reading_from_stream(save_path, duration, num_files_to_create)
        self._close_recording_resources()
        print("Stop recording")

    def create_current_wav_file(self):
        
        self._counter = self._counter + 1 
        the_time  = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S%z")
        ctr_str   = f'{self._counter:04}'
        save_path = self.save_path.replace(".wav",f"_C_{ctr_str}_D_{the_time}.wav")        
        self._create_wav_file(save_path)
    
    def _create_recording_resources(self) -> None:
        self._pyaudio = pyaudio.PyAudio()
        self._stream = self._pyaudio.open(**self.stream_params.to_dict())
        self.create_current_wav_file()

    def _create_wav_file(self, save_path: str):
        print(f"creating new wav: {save_path}")
        self._wav_file = wave.open(save_path, "wb")
        self._wav_file.setnchannels(self.stream_params.channels)
        self._wav_file.setsampwidth(self._pyaudio.get_sample_size(self.stream_params.format))
        self._wav_file.setframerate(self.stream_params.rate)


    def close_current_wav_file(self, wav_file) -> None:
        wav_file.close()

    def _write_wav_file_reading_from_stream(self, save_path: str, duration: int, num_files_to_create: int) -> None:
        with ThreadPoolExecutor(max_workers = 5) as executor:
            for i in range(num_files_to_create):
                for _ in range(int(self.stream_params.rate * duration / self.stream_params.frames_per_buffer)):
                    audio_data = self._stream.read(self.stream_params.frames_per_buffer)
                    self._wav_file.writeframes(audio_data)
                    
                executor.submit(self.close_current_wav_file, self._wav_file)
                if i < num_files_to_create-1:
                    self.create_current_wav_file()

    def _close_recording_resources(self) -> None:
        self._stream.close()
        self._pyaudio.terminate()


In [ ]:
stream_params      = StreamParams()
stream_params.rate = 16000
recorder           = Recorder(stream_params)
recorder.record(60, "/home/amitli/Datasets/speakathon/audio.wav", num_files_to_create=60*18)

<h1 style="background-color:#3cA8EF;"> <center><a id='ParseRecords'></a>  Parse recordings </center> </h1>

In [ ]:
def get_sorted_files(files_path):
    all_files = glob.glob(files_path)
    all_files.sort()    
    return all_files    

In [ ]:
NIGHT_RUN_INPUT_FOLDER = r"/home/amitli/Debug/25_26_jul/Night_25_26/*"

In [ ]:
all_files = get_sorted_files(NIGHT_RUN_INPUT_FOLDER)
one_file  = AudioSegment.from_wav(all_files[0]) 
for i in range(1, len(all_files)):
    one_file = one_file + AudioSegment.from_wav(all_files[i])   

In [ ]:
NIGHT_RUN_1_ONE_FILE = r"/home/amitli/Debug/25_26_jul/One_file/One_file.wav"

In [ ]:
one_file.export(NIGHT_RUN_1_ONE_FILE, format="wav")

In [ ]:
arr_start_speech_time[:5]

In [ ]:
#AudioSegment.from_wav(r"/home/amitli/Debug/25_26_jul/Night_25_26/audio_C_0001_D_25_07_2023_15_18_31.wav")[9.9*1000:11*1000]

In [ ]:
full_wav              = AudioSegment.from_wav(NIGHT_RUN_1_ONE_FILE)
mp3_len               = arr_time
num_of_speakers       = len(set(arr_clients))
num_of_mp3_per_speker = arr_num_cv_per_speaker


start_rec_time_ms = int(9.9*1000) - arr_start_speech_time[0] * 1000
arr_results       = []
counter           = 0
end               = 0

for i_speaker in range(num_of_speakers):
   
    for i_mp3 in range(num_of_mp3_per_speker[i_speaker]):
                
        # --- speech
        start = start_rec_time_ms + arr_start_speech_time[counter]*1000
        #end   = arr_end_speech_time[counter]*1000 + start_rec_time_ms
        end   = start + mp3_len[counter]*1000   
        
        
        if end > len(full_wav):
            break
                   
        arr_results.append(full_wav[start-300 : end])                   
        counter = counter + 1
           

print(f"Finished at = {end} / {len(full_wav)}, Found: {len(arr_results)}")

In [ ]:
test = 2354
arr_results[test]

In [ ]:
print(f"{arr_cv[test]}, {arr_time[test]}, recorded: {arr_results[test].duration_seconds}")
AudioSegment.from_mp3(f"{ARABIC_PATH}/clips/{arr_cv[test]}")

In [ ]:
last_speaker  = 0
export_path   = "/home/amitli/Debug/25_26_jul/Outputs_wav"
full_path     = ""
for i in tqdm(range(2993)):
    
    speaker     = arr_clients[i]
    if speaker != last_speaker:
        expoer_folder = speaker
        full_path     = f"{export_path}/{speaker}"
        if not os.path.exists(full_path):  
            os.makedirs(full_path)        
        
    last_speaker = speaker
    tactic_file  = arr_results[i]
    cv_name      = arr_cv[i]
    lang         = arr_lang[i]
    
    file_name    = f"tactic_{lang}_{cv_name}"
    #arr_results[i].export(f"{full_path}/{file_name}", format="mp3")
    arr_results[i].export(f"{full_path}/{file_name[:-4]}.wav", format="wav")    

<h1 style="background-color:#3cA8EF;"> <center> <a id='Plot_dBFS'></a>  Plot (dBFS) </center> </h1>

In [ ]:
import plotly.graph_objects as go
import numpy as np
import wave

def read_wav_file(file_path):
    with wave.open(file_path, "rb") as wav_file:
        num_frames   = wav_file.getnframes()
        sample_width = wav_file.getsampwidth()
        sample_rate  = wav_file.getframerate()
        audio_data   = np.frombuffer(wav_file.readframes(num_frames), dtype=np.int16) 
    return audio_data, sample_rate


def convert_to_dbfs(audio_data):
    max_value = np.max(np.abs(audio_data))
    dbfs = 20 * np.log10(audio_data.astype(np.float32) / max_value)
    return dbfs


def plot_dBFS(audio_data, sample_rate):
    duration = len(audio_data) / sample_rate
    time = np.linspace(0, duration, len(audio_data))
    dbfs = convert_to_dbfs(audio_data)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time, y=dbfs, mode="lines"))
    fig.update_layout(
        title="dBFS Plot",
        xaxis_title="Time (s)",
        yaxis_title="dBFS",
    )
    return fig

    
file_path = r"/home/amitli/Debug/25_26_jul/Night_25_26/audio_C_0001_D_25_07_2023_15_18_31.wav"
audio_data, sample_rate = read_wav_file(file_path)
fig                     = plot_dBFS(audio_data, sample_rate)
fig.show()    

[Go to start](#start_cell) </br>
[Parse log](#parseLog) </br>
